We need a Bot to handle Technical questons related to various different areas such as ML, Java, C# etc. and each area should be handled by different LLMs. Bot should be able answer questions in simple or advanced terms depending on user's backgroud
 - To solve this we may have to use LLMRouterChain
 - As first iteration just take care of Machine Learing area
 - Workflow - INPUT --> ROUTER --> ROUTER DECIDES CHAIN --> CHAIN --> OUTPUT  



In [ ]:
!pip install openai
!pip install --upgrade langchain

In [ ]:
import langchain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate
from langchain.chains import SequentialChain, LLMChain
import os
chat_llm = ChatOpenAI()

In [ ]:
ml_simple_template = '''You are a Machine Learning expert who is really focused on beginners and explain complex concepts in simple to understand terms. You assume to prior knowledge. Here is your question:\n{input}'''

In [ ]:
ml_advanced_template = '''You are a Machine Learning expert who explain machine learing concepts to advanced audience members. You can assume anyone you answer has expert knowledge in Machine Learning. Here is your question:\n{input}'''

In [ ]:
prompt_infos =[
    {
        'name': 'beginner Machine Learning',
        'description': 'Answers basic Machine Learning questions',
        'template': ml_simple_template
    },

    {
        'name': 'advanced Machine Learning',
        'description': 'Answers advanced Machine Learning questions',
        'template': ml_advanced_template
    }

]

destination_chains={}
for p_info in prompt_infos:
    name = p_info['name']
    description = p_info['description']
    template = p_info['template']
    prompt = ChatPromptTemplate.from_template(template = template)
    chain = LLMChain(llm=chat_llm, prompt=prompt)
    destination_chains[name] = chain




In [9]:
default_prompt = ChatPromptTemplate.from_template('{input}')
default_chain = LLMChain(llm=chat_llm, prompt=default_prompt)


In [10]:
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [15]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [12]:
destinations = [ f"{p['name']}:{p['description']}" for p in prompt_infos]


In [14]:
destinations

['beginner Machine Learning:Answers basic Machine Learning questions',
 'advanced Machine Learning:Answers advanced Machine Learning questions']

In [16]:
destination_str = "\n".join(destinations)
print(destination_str)


beginner Machine Learning:Answers basic Machine Learning questions
advanced Machine Learning:Answers advanced Machine Learning questions


In [17]:
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destination_str)

print(router_template)




Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
beginner Machine Learning:Answers basic Machine Learning questions
advanced Machine Learning:Answers 

In [18]:
router_prompt = PromptTemplate(template = router_template,
                               input_variables=['input'],
                               output_parser=RouterOutputParser())

In [20]:
from langchain.chains.router import MultiPromptChain
router_chain = LLMRouterChain.from_llm(chat_llm, router_prompt)
chain = MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True)



In [21]:
chain.run("What is BackPropogation?")



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


beginner Machine Learning: {'input': 'What is Backpropagation?'}
> Finished chain.


"Backpropagation is a fundamental concept in Machine Learning, specifically in training neural networks. It is a mathematical algorithm that allows us to adjust the weights and biases of the neural network to minimize the difference between the predicted output and the actual output.\n\nTo understand backpropagation, let's consider a simplified analogy. Imagine you have a friend who is learning how to play a video game for the first time. Initially, your friend is terrible at the game, but they want to improve. So, they start playing and you watch their performance closely.\n\nAfter each level, you provide feedback to your friend on their mistakes and guide them on what they should have done differently. Gradually, your friend learns from their mistakes and gets better at the game.\n\nIn this analogy, your friend represents the neural network, and you represent the backpropagation algorithm. The aim is to guide the neural network towards making better predictions by adjusting its param

In [23]:
chain.run("Explain Backpropodation with Mathematical equations?")



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:278: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


advanced Machine Learning: {'input': 'Explain Backpropagation with Mathematical equations?'}
> Finished chain.


"Backpropagation is a widely used algorithm in the field of deep learning for training artificial neural networks. It is based on the chain rule of calculus and allows us to efficiently compute the gradients of the weights in a neural network.\n\nLet's start by considering a simple feedforward neural network with a single hidden layer. We'll denote the input to the network as x, the hidden layer activation as h, the output layer activation as y, the weights between the input and hidden layer as W, and the weights between the hidden and output layer as V. Additionally, we'll assume that we have a target output for a given input, denoted as t.\n\nThe forward pass of the neural network can be expressed as follows:\n\nh = f(Wx)    (1)\ny = g(Vh)     (2)\n\nHere, f() and g() represent the activation functions applied to the hidden and output layers, respectively.\n\nTo train the neural network, we need to update the weights W and V such that the network's output y gets closer to the target 